## Project to create two maps - marker and cluster_marker, showing each bike rack in Davidson County.

- This was a practice to get markers on a map, so the following issues were not resolved:
    - Duplicate values were not removed from the data, but several exist which appear to be duplicated by name, but not by exact lat/long coordinates (though they are very close to each other)
    - The racks are 'Existing', 'Proposed', 'Future Artistic' (status column). All are shown in these maps with blue markers, but could be shown with three different colors.    

In [1]:
from shapely.geometry import Point   #This is to pull Point module (could import ,Line after Point if wanted)
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [3]:
# Using geopandas to read in the file
racks = gpd.read_file('../data/Bike Racks (GIS).geojson')  #Using gpd geopandas to read in the file
print(racks.crs)
racks.head(3)

{'init': 'epsg:4326'}


,detail_loc,quantity,location,domain,idnumber,objectid,status,lon,type,data_colle,lat,capacity,geometry
0,Hickory Hollow Mall,1,Mall,Public,113,1,Existing,-86.6554527779,Rack,Health Dept,36.0499277778,12,POINT (-86.65545 36.04993)
1,Stones River Greenway - Donelson/Hermitage Area,2,Greenway,Public,114,2,Existing,-86.6340625226,Rack,Health Dept,36.1852800378,4,POINT (-86.63406 36.18528)
2,"Percy Priest Reservoir, Stones River Greenway",0,Park/Greenway,Public,256,3,Proposed,-86.6206705697,Rack,Health Dept,36.1584215986,0,POINT (-86.62067 36.15842)


In [4]:
# Changing data type to EPSG:4326
racks.crs = "EPSG:4326"
print(racks.crs)

EPSG:4326


In [9]:
#Dropped the 4 rows that has NaN values in lat/lon
#Showing .info to confirm that lon and lat are now floats and only 426 rows, 
#    with no non-null values.
racks = racks.dropna(subset=['lat'])
racks['lat'] = racks.lat.astype(float)
racks['lon'] = racks.lon.astype(float)
racks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 426 entries, 0 to 429
Data columns (total 13 columns):
detail_loc    426 non-null object
quantity      426 non-null object
location      426 non-null object
domain        426 non-null object
idnumber      426 non-null object
objectid      426 non-null object
status        426 non-null object
lon           426 non-null float64
type          426 non-null object
data_colle    426 non-null object
lat           426 non-null float64
capacity      426 non-null object
geometry      426 non-null geometry
dtypes: float64(2), geometry(1), object(10)
memory usage: 46.6+ KB


In [10]:
#Creating subset of Existing racks, to show via different color later. Checking file type.
existing = racks[racks.status == 'Existing']
print(existing.crs)
existing.head(1)

EPSG:4326


,detail_loc,quantity,location,domain,idnumber,objectid,status,lon,type,data_colle,lat,capacity,geometry
0,Hickory Hollow Mall,1,Mall,Public,113,1,Existing,-86.655453,Rack,Health Dept,36.049928,12,POINT (-86.65545 36.04993)


In [11]:
#Creating subset of Existing racks, to show via different color later. Checking file type.
proposed = racks[racks.status == 'Proposed']
print(proposed.crs)
proposed.head(1)

EPSG:4326


,detail_loc,quantity,location,domain,idnumber,objectid,status,lon,type,data_colle,lat,capacity,geometry
2,"Percy Priest Reservoir, Stones River Greenway",0,Park/Greenway,Public,256,3,Proposed,-86.620671,Rack,Health Dept,36.158422,0,POINT (-86.62067 36.15842)


In [12]:
#Creating subset of Existing racks, to show via different color later. Checking file type.
future_artistic = racks[racks.status == 'Future Artistic']
print(future_artistic.crs)
future_artistic.head(1)

EPSG:4326


,detail_loc,quantity,location,domain,idnumber,objectid,status,lon,type,data_colle,lat,capacity,geometry
9,"Bus Stop; Church St at 5th Ave N, Fifth Third ...",0,Bus Stop,Public,83,10,Future Artistic,-86.779955,Rack,Health Dept,36.163372,0,POINT (-86.77995 36.16337)


In [ ]:
#Confirmed total 426 entries, with lat & lon both float
# print(proposed.info())
# print(existing.info())
# print(future_artistic.info())

# Next:  create map with diff color maker for each of 3 df's, then add polygon of parks to map (on top of Nashville - see code from Sophia)...  DOWNLOAD PARKS a second time. Current file corrupted.

In [ ]:
map_racks = folium.Map(
    location = [36.174465, -86.767960],    #Nashville coordinates
    zoom_start = 12
)

for row_index, row_values in racks.iterrows():
    loc = [row_values['lat'], row_values['lon']]
    pop = str(row_values['detail_loc'])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(map_racks)
map_racks.save('../maps/map_racks.html')   #Tell it the path to save the map to (make sure we have a maps folder)

#display our map
map_racks

In [ ]:
culster_map_racks = folium.Map(
    location = [36.174465, -86.767960],    #Nashville coordinates
    zoom_start = 12
)

#create a marker cluster, and adds it to the map.
marker_cluster = MarkerCluster().add_to(culster_map_racks)

# inside the loop add each marker to the cluster
for row_index, row_values in racks.iterrows():
    loc = [row_values['lat'], row_values['lon']]
    pop = str(row_values['detail_loc'])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(marker_cluster)
    
culster_map_racks.save('../maps/culster_map_racks.html')   #Tell it the path to save the map to (make sure we have a maps folder)

#display our map
culster_map_racks